In [ ]:
%matplotlib inline
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import edward as ed
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap, Normalize
from tempfile import NamedTemporaryFile
import seaborn as sns
import numpy as np
import six
import tensorflow as tf

plt.style.use('seaborn-talk')
sns.set_context("talk", font_scale=1.4)
sess = ed.get_session()

sns.palplot(sns.color_palette())

In [ ]:
# this can be done only before using Edward
ed.set_seed(42)

# Bayesian Linear Regression

In [ ]:
from edward.models import Normal

In [ ]:
x = tf.range(-5.0, 5.0, 0.001)
plt.plot(*sess.run([x, Normal(loc=tf.ones(1) * 0.0,                # blue
                              scale=tf.ones(1) * 1.0).prob(x)]));
plt.plot(*sess.run([x, Normal(loc=tf.ones(1) * 2.0,                # green
                              scale=tf.ones(1) * 1.0).prob(x)]));
plt.plot(*sess.run([x, Normal(loc=tf.ones(1) * 0.0,                # red
                              scale=tf.ones(1) * 2.0).prob(x)]));

In [ ]:
# DATA
N1 = 10  # number of training data points in first batch
N2 = 90  # number of training data points in second batch
Np = 10  # number of test data points
D = 1  # number of features

weights_true = sess.run(Normal(loc=tf.ones(D) * 2.0,
                               scale=tf.ones(D) * 0.1))  # unknown true weights
intercept_true = sess.run(Normal(loc=tf.zeros(1),
                                 scale=tf.ones(1)))  # unknown true intercept
noise_true = 0.35  # unknown true amount of noise

def build_dataset(N):
    x = Normal(loc=tf.zeros([N, D]), scale=tf.ones([N, D]))
    y = Normal(loc=ed.dot(x, weights_true) + intercept_true, scale=noise_true)
    return sess.run([x, y])

x_train1, y_train1 = build_dataset(N1)
x_train2, y_train2 = build_dataset(N2)
x_test, y_test = build_dataset(Np)

In [ ]:
plt.scatter(x_train1, y_train1, s=20.0);  # blue
# plt.scatter(x_train2, y_train2, s=20.0);  # green
plt.scatter(x_test, y_test, s=20.0,
            color=sns.color_palette().as_hex()[2]);  # red

#### Little Noise

In [ ]:
from edward.models import Normal

In [ ]:
# FORWARD MODEL
x = tf.placeholder(tf.float32, [N1, D])
weights = Normal(loc=tf.zeros(D), scale=tf.ones(D))
intercept = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(x, weights) + intercept,
           scale=tf.ones(N1) * 0.01)  # with little noise

In [ ]:
# BACKWARD MODEL
q_weights = Normal(loc=tf.Variable(tf.random_normal([D])),
                   scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
q_intercept = Normal(loc=tf.Variable(tf.random_normal([1])),
                     scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [ ]:
# INFERENCE
inference = ed.KLqp(latent_vars={weights: q_weights,
                                 intercept: q_intercept},
                    data={x: x_train1,
                          y: y_train1})
inference.run(n_samples=50, n_iter=1000)

In [ ]:
# CRITICISM
plt.scatter(x_train1, y_train1, s=20.0);  # blue
plt.scatter(x_test, y_test, s=20.0,
            color=sns.color_palette().as_hex()[2]);  # red

xp = tf.placeholder(tf.float32, [2, D])
[plt.plot(np.linspace(-4.0, 4.0, 2),
          sess.run(ed.dot(xp, q_weights) + q_intercept,
                   {xp: np.linspace(-4.0, 4.0, 2)[:, np.newaxis]}),
          color='black', alpha=0.1)
 for _ in range(50)];

In [ ]:
y_post = ed.copy(y, {weights: q_weights,
                     intercept: q_intercept})
# this is equivalent to
# y_post = Normal(loc=ed.dot(x, q_weights) + q_intercept,
#                 scale=tf.ones(N1) * 0.01)
# ed.copy works for us only because Np=N1!

In [ ]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={x: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={x: x_test, y_post: y_test}))

In [ ]:
# that's not bad, but the model is way too overconfident.

#### More Noise

In [ ]:
# FORWARD MODEL
x = tf.placeholder(tf.float32, [N1, D])
weights = Normal(loc=tf.zeros(D), scale=tf.ones(D))
intercept = Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = Normal(loc=ed.dot(x, weights) + intercept,
           scale=tf.ones(N1))  # with more noise

In [ ]:
# BACKWARD MODEL
q_weights = Normal(loc=tf.Variable(tf.random_normal([D])),
                   scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
q_intercept = Normal(loc=tf.Variable(tf.random_normal([1])),
                     scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))

In [ ]:
# INFERENCE
inference = ed.KLqp(latent_vars={weights: q_weights,
                                 intercept: q_intercept},
                    data={x: x_train1,
                          y: y_train1})
inference.run(n_samples=50, n_iter=1000)

In [ ]:
# CRITICISM
plt.scatter(x_train1, y_train1, s=20.0);  # blue
plt.scatter(x_test, y_test, s=20.0,
            color=sns.color_palette().as_hex()[2]);  # red

xp = tf.placeholder(tf.float32, [2, D])
[plt.plot(np.linspace(-4.0, 4.0, 2),
          sess.run(ed.dot(xp, q_weights) + q_intercept,
                   {xp: np.linspace(-4.0, 4.0, 2)[:, np.newaxis]}),
          color='black', alpha=0.1)
 for _ in range(50)];

In [ ]:
y_post = ed.copy(y, {weights: q_weights,
                     intercept: q_intercept})
# this is equivalent to
# y_post = Normal(loc=ed.dot(x, q_weights) + q_intercept,
#                 scale=tf.ones(N1))
# ed.copy works for us only because Np=N1!

In [ ]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={x: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={x: x_test, y_post: y_test}))

In [ ]:
# too much noise!
# the model could be more confident.
# what is the right amount of noise?
# what do we do in these cases?
# we put a prior on the noise.

#### Prior On Noise

In [ ]:
from edward.models import InverseGamma

In [ ]:
x = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([x, InverseGamma(concentration=5.0, rate=1.0).prob(x)]));  # blue
plt.plot(*sess.run([x, InverseGamma(concentration=3.0, rate=1.0).prob(x)]));  # green
plt.plot(*sess.run([x, InverseGamma(concentration=1.0, rate=1.0).prob(x)]));  # red
plt.axvline(x=noise_true**2);  # blue

In [ ]:
# FORWARD MODEL
x = tf.placeholder(tf.float32, [N1, D])
weights = Normal(loc=tf.zeros(D), scale=tf.ones(D))
intercept = Normal(loc=tf.zeros(1), scale=tf.ones(1))
var = InverseGamma(concentration=5.0, rate=1.0)  # noise prior
y = Normal(loc=ed.dot(x, weights) + intercept,
           scale=tf.ones(N1) * tf.sqrt(var))

In [ ]:
# BACKWARD MODEL
q_weights = Normal(loc=tf.Variable(tf.random_normal([D])),
                   scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
q_intercept = Normal(loc=tf.Variable(tf.random_normal([1])),
                     scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))
q_var = InverseGamma(concentration=tf.nn.softplus(tf.Variable(tf.random_normal([]))),
                     rate=tf.nn.softplus(tf.Variable(tf.random_normal([]))))

In [ ]:
# INFERENCE
inference = ed.KLqp(latent_vars={weights: q_weights,
                                 intercept: q_intercept,
                                 var: q_var},
                    data={x: x_train1,
                          y: y_train1})
inference.run(n_samples=50, n_iter=1000)

In [ ]:
# CRITICISM
xp = tf.range(0.0, 1.0, 0.001)
plt.plot(*sess.run([xp, q_var.prob(xp)]));
plt.axvline(x=noise_true**2);

In [ ]:
xp = tf.range(0.0, 4, 0.001)
plt.plot(*sess.run([xp, q_weights.prob(xp)]));
plt.axvline(x=weights_true);

In [ ]:
xp = tf.range(-4, 0, 0.001)
plt.plot(*sess.run([xp, q_intercept.prob(xp)]));
plt.axvline(x=intercept_true);

In [ ]:
plt.scatter(x_train1, y_train1, s=20.0);  # blue
plt.scatter(x_test, y_test, s=20.0,
            color=sns.color_palette().as_hex()[2]);  # red

xp = tf.placeholder(tf.float32, [2, D])
[plt.plot(np.linspace(-4.0, 4.0, 2),
          sess.run(ed.dot(xp, q_weights) + q_intercept,
                   {xp: np.linspace(-4.0, 4.0, 2)[:, np.newaxis]}),
          color='black', alpha=0.1)
 for _ in range(50)];

In [ ]:
y_post = ed.copy(y, {weights: q_weights,
                     intercept: q_intercept,
                     var: q_var})
# this is equivalent to
# y_post = Normal(loc=ed.dot(x, q_weights) + q_intercept,
#                 scale=tf.ones(N1) * tf.sqrt(q_var))
# ed.copy works for us only because Np=N1!

In [ ]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={x: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={x: x_test, y_post: y_test}))

#### Use The Posterior for Batch 1 as The Prior for Batch 2

In [ ]:
# FORWARD MODEL FOR 2nd BATCH
x = tf.placeholder(tf.float32, [N2, D])
weights = q_weights
intercept = q_intercept
var = q_var
y = Normal(loc=ed.dot(x, weights) + intercept, scale=tf.ones(N2) * tf.sqrt(var))

In [ ]:
# BACKWARD MODEL FOR 2nd BATCH
q_weights2 = Normal(loc=tf.Variable(tf.random_normal([D])),
                    scale=tf.nn.softplus(tf.Variable(tf.random_normal([D]))))
q_intercept2 = Normal(loc=tf.Variable(tf.random_normal([1])),
                      scale=tf.nn.softplus(tf.Variable(tf.random_normal([1]))))
q_var2 = InverseGamma(concentration=tf.nn.softplus(tf.Variable(tf.random_normal([]))),
                      rate=tf.nn.softplus(tf.Variable(tf.random_normal([]))))

In [ ]:
# INFERENCE FOR 2nd BATCH
inference = ed.KLqp(latent_vars={weights: q_weights2,
                                 intercept: q_intercept2,
                                 var: q_var2},
                    data={x: x_train2,
                          y: y_train2})
inference.run(n_samples=50, n_iter=1000)

In [ ]:
# CRITICISM FOR 2nd BATCH
plt.scatter(np.concatenate((x_train1, x_train2)),
            np.concatenate((y_train1, y_train2)), s=20.0);  # blue
plt.scatter(x_test, y_test, s=20.0,
            color=sns.color_palette().as_hex()[2]);  # red

xp = tf.placeholder(tf.float32, [2, D])
[plt.plot(np.linspace(-4.0, 4.0, 2),
          sess.run(ed.dot(xp, q_weights2) + q_intercept2,
                   {xp: np.linspace(-4.0, 4.0, 2)[:, np.newaxis]}),
          color='black', alpha=0.1)
 for _ in range(50)];

In [ ]:
xp = tf.placeholder(tf.float32, [Np, D])
y_post = Normal(loc=ed.dot(xp, q_weights2) + q_intercept2,
                scale=tf.ones(Np) * tf.sqrt(q_var2))

In [ ]:
print("Mean squared error on test data:")
print(ed.evaluate('mean_squared_error', data={xp: x_test, y_post: y_test}))

print("Mean absolute error on test data:")
print(ed.evaluate('mean_absolute_error', data={xp: x_test, y_post: y_test}))